Imports

In [ ]:
import pandas as pd
import seaborn as sns
import numpy as np
from matplotlib import pyplot as plt

Initial Prep

In [ ]:
df = pd.read_csv('https://covid.ourworldindata.org/data/owid-covid-data.csv')

df = df[df['iso_code'].notna()] # drop NaN locations
df = df[df['iso_code'] != 'OWID_WRL'] # and World data

df['date']= pd.to_datetime(df['date']) # make date datetime
df.continent = df.continent.astype('category') # make continent datetime
df.location = df.location.astype('category') # make country category
# df['location'].cat.categories

# Delete rows with total cases = 0
indexNames = df[ df['total_cases'] == 0 ].index
df.drop(indexNames , inplace=True)
df.reset_index(inplace=True, drop=True)
df.head()

,iso_code,continent,location,date,total_cases,new_cases,total_deaths,new_deaths,total_cases_per_million,new_cases_per_million,total_deaths_per_million,new_deaths_per_million,new_tests,total_tests,total_tests_per_thousand,new_tests_per_thousand,new_tests_smoothed,new_tests_smoothed_per_thousand,tests_units,stringency_index,population,population_density,median_age,aged_65_older,aged_70_older,gdp_per_capita,extreme_poverty,cardiovasc_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy
0,ABW,North America,Aruba,2020-03-13,2.0,2.0,0.0,0.0,18.733,18.733,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00,106766.0,584.8,41.2,13.085,7.452,35973.781,NaN,NaN,11.62,NaN,NaN,NaN,NaN,76.29
1,ABW,North America,Aruba,2020-03-20,4.0,2.0,0.0,0.0,37.465,18.733,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,33.33,106766.0,584.8,41.2,13.085,7.452,35973.781,NaN,NaN,11.62,NaN,NaN,NaN,NaN,76.29
2,ABW,North America,Aruba,2020-03-24,12.0,8.0,0.0,0.0,112.395,74.930,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,44.44,106766.0,584.8,41.2,13.085,7.452,35973.781,NaN,NaN,11.62,NaN,NaN,NaN,NaN,76.29
3,ABW,North America,Aruba,2020-03-25,17.0,5.0,0.0,0.0,159.227,46.831,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,44.44,106766.0,584.8,41.2,13.085,7.452,35973.781,NaN,NaN,11.62,NaN,NaN,NaN,NaN,76.29
4,ABW,North America,Aruba,2020-03-26,19.0,2.0,0.0,0.0,177.959,18.733,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,44.44,106766.0,584.8,41.2,13.085,7.452,35973.781,NaN,NaN,11.62,NaN,NaN,NaN,NaN,76.29


Filling in NaNs

In [ ]:
# Too many nans
df.drop(inplace=True, columns=["new_tests", "total_tests", "total_tests_per_thousand", "new_tests_per_thousand", "new_tests_smoothed", "new_tests_smoothed_per_thousand", "tests_units", "handwashing_facilities"])
# Dropping rows with too much missing data on total_cases and more
df = df[df['total_cases'].notna()]
# Fills in nans by category - soo cooool! Could definitely do this cleaner but eh.
df.stringency_index = df.groupby('location')['stringency_index'].apply(lambda x: x.fillna(x.mean()))
df.stringency_index = df.stringency_index.fillna(df.stringency_index.mean())
df.population_density = df.groupby('location')['population_density'].apply(lambda x: x.fillna(x.mean()))
df.population_density = df.population_density.fillna(df.population_density.mean())
df.median_age = df.groupby('location')['median_age'].apply(lambda x: x.fillna(x.mean()))
df.median_age = df.median_age.fillna(df.median_age.mean())
df.aged_65_older = df.groupby('location')['aged_65_older'].apply(lambda x: x.fillna(x.mean()))
df.aged_65_older = df.aged_65_older.fillna(df.aged_65_older.mean())
df.aged_70_older = df.groupby('location')['aged_70_older'].apply(lambda x: x.fillna(x.mean()))
df.aged_70_older = df.aged_70_older.fillna(df.aged_70_older.mean())
df.gdp_per_capita = df.groupby('location')['gdp_per_capita'].apply(lambda x: x.fillna(x.mean()))
df.gdp_per_capita = df.gdp_per_capita.fillna(df.gdp_per_capita.mean())
df.extreme_poverty = df.groupby('location')['extreme_poverty'].apply(lambda x: x.fillna(x.mean()))
df.extreme_poverty = df.extreme_poverty.fillna(df.extreme_poverty.mean())
df.cardiovasc_death_rate = df.groupby('location')['cardiovasc_death_rate'].apply(lambda x: x.fillna(x.mean()))
df.cardiovasc_death_rate = df.cardiovasc_death_rate.fillna(df.cardiovasc_death_rate.mean())
df.diabetes_prevalence = df.groupby('location')['diabetes_prevalence'].apply(lambda x: x.fillna(x.mean()))
df.diabetes_prevalence = df.diabetes_prevalence.fillna(df.diabetes_prevalence.mean())
df.female_smokers = df.groupby('location')['female_smokers'].apply(lambda x: x.fillna(x.mean()))
df.female_smokers = df.female_smokers.fillna(df.female_smokers.mean())
df.male_smokers = df.groupby('location')['male_smokers'].apply(lambda x: x.fillna(x.mean()))
df.male_smokers = df.male_smokers.fillna(df.male_smokers.mean())
df.hospital_beds_per_thousand = df.groupby('location')['hospital_beds_per_thousand'].apply(lambda x: x.fillna(x.mean()))
df.hospital_beds_per_thousand = df.hospital_beds_per_thousand.fillna(df.hospital_beds_per_thousand.mean())
df.life_expectancy = df.groupby('location')['life_expectancy'].apply(lambda x: x.fillna(x.mean()))
df.life_expectancy = df.life_expectancy.fillna(df.life_expectancy.mean())

# Check columns missing value %
percent_missing = df.isnull().sum() * 100 / len(df)
missing_value_df = pd.DataFrame({'column_name': df.columns,
                                 'percent_missing': percent_missing})
# missing_value_df # see the percentage of missing values

# df["new_tests"] = df.groupby("location").transform(lambda x: x.fillna(x.mean())) # Fill NaN by mean in each group
# df["total_tests"] = df.groupby("location").transform(lambda x: x.fillna(x.mean())) # Fill NaN by mean in each group

missing_value_df


,column_name,percent_missing
iso_code,iso_code,0.0
continent,continent,0.0
location,location,0.0
date,date,0.0
total_cases,total_cases,0.0
new_cases,new_cases,0.0
total_deaths,total_deaths,0.0
new_deaths,new_deaths,0.0
total_cases_per_million,total_cases_per_million,0.0
new_cases_per_million,new_cases_per_million,0.0


Set Target and Features

In [ ]:
target_name = 'new_deaths'
# sibsp stands for the number of sibilings and parents on board
feature_names = ['total_cases', 'new_cases',
       'total_deaths', 'total_cases_per_million',
       'new_cases_per_million', 'total_deaths_per_million',
       'new_deaths_per_million', 'stringency_index', 'population',
       'population_density', 'median_age', 'aged_65_older', 'aged_70_older',
       'gdp_per_capita', 'extreme_poverty', 'cardiovasc_death_rate',
       'diabetes_prevalence', 'female_smokers', 'male_smokers',
       'hospital_beds_per_thousand', 'life_expectancy']
X, y = df[feature_names], df[target_name]

Test-Train Split

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)
print(f"Average new deaths in all data {round(y.mean(), 2)}")
print(f"Average new deaths in the test data {round(y_test.mean(), 2)}")
print(f"Average new deaths in the train data {round(y_train.mean(), 2)}")

Average new deaths in all data 22.27
Average new deaths in the test data 23.64
Average new deaths in the train data 21.82


Train Model

In [ ]:
print('Training Features Shape:', X_train.shape)
print('Training Labels Shape:', y_train.shape)
print('Testing Features Shape:', X_test.shape)
print('Testing Labels Shape:', y_test.shape)

# Import the model we are using
from sklearn.ensemble import RandomForestRegressor
# Instantiate model with 1000 decision trees
rf = RandomForestRegressor(n_estimators = 1000, random_state = 42)
# Train the model on training data
rf.fit(X_train, y_train);

Training Features Shape: (22671, 21)
Training Labels Shape: (22671,)
Testing Features Shape: (7557, 21)
Testing Labels Shape: (7557,)


Test Model

In [ ]:
# Use the forest's predict method on the test data
predictions = rf.predict(X_test)
# Calculate the absolute errors
errors = abs(predictions - y_test)
# Print out the mean absolute error (mae)
print('Mean Absolute Error:', round(np.mean(errors), 2), 'degrees.')

Mean Absolute Error: 2.39 degrees.


Predict Wayca Deaths

In [ ]:
wayca_test = pd.DataFrame (
    {
        'total_cases': [175494], 'new_cases': [3360], 'total_deaths': [2238], 'total_cases_per_million': [1065.61],
        'new_cases_per_million': [20.402], 'total_deaths_per_million': [13589],
        'new_deaths_per_million': [0.249], 'stringency_index': [74.54], 'population': [1.64689e+08],
        'population_density': [1265.04], 'median_age': [27.5], 'aged_65_older': [5.098], 'aged_70_older': [3.262],
        'gdp_per_capita': [3523.98], 'extreme_poverty': [14.8], 'cardiovasc_death_rate': [298.003],
        'diabetes_prevalence': [8.38], 'female_smokers': [1], 'male_smokers': [44.7],
        'hospital_beds_per_thousand': [0.8], 'life_expectancy': [72.59]
    })

# Use the forest's predict method on the test data
wayca_predict = rf.predict(wayca_test)
print("New Deaths in Wayca: ", wayca_predict[0])


New Deaths in Wayca:  38.132
